In [2]:
import transformer as kernel_transformer

from torch.utils.cpp_extension import load
import torch

from matplotlib import pyplot as plt
trampoline = load(
    "extension",
    sources=["trampoline.cu"],
    extra_ldflags=["-lnvrtc", "-lcuda"],
    extra_cuda_cflags=["-g"],
)


/home/normal/.local/lib/python3.12/site-packages/torch/utils/cpp_extension.py:1967: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


In [3]:

#ptx.shfl.sync.down.b32(d, a, i, 31, 0xFFFFFFFF) 

@kernel_transformer.fragment()
def warp_sum(out: "u32", a: "u32"):
    accum: u32 = a 
    d: u32
    i = 16
    while i > 0:
        ptx.shfl.sync.down.b32(d, accum, i, 31, 0xFFFFFFFF) 
        accum += d
        i //= 2
    if u32("%laneid") == 0:
        out = accum

@kernel_transformer.kernel()
def shfl_test(buffer: "u64"):
    a: u32 = u32("%laneid")
    result: u32
    warp_sum(kernel_builder, result, a)
    
    
    buffer_global: u64
    
    tidx: u32 = u32("%tid.x") + u32("%ntid.x") * u32("%ctaid.x")
    buffer += tidx * 4
    
    ptx.cvta.to._global.u64(buffer_global, buffer)
    if u32("%laneid") == 0:
        ptx.st._global.u32([buffer_global], result)

In [4]:
kernel_builder = kernel_transformer.KernelBuilder()
shfl_test(kernel_builder)
kernel_code = kernel_builder.generate("sm_75")

result = torch.zeros((64,), dtype=torch.uint32, device="cuda")

wrapper = trampoline.CuModuleWrapper()
wrapper.load_ptx_code(kernel_code)

wrapper.launch_kernel("shfl_test", (1, 1, 1), (64, 1, 1), (result.data_ptr(),))
print(result.cpu())

tensor([496,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0, 496,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0], dtype=torch.uint32)
